<a href="https://www.kaggle.com/code/handandegerli/online-retail-arb-recommender-system?scriptVersionId=182830447" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### A Product Recommendation System with Association Rule Learning
---

**Business Problem:** Product recommendations are requested based on the basket information of 3 different users.Utilize association rules to generate the most suitable product recommendations, which can be one or more products.Derive the decision rules from the 2010-2011 data of customers in Germany.\
Product ID in User 1's basket: 21987\
Product ID in User 2's basket: 23235\
Product ID in User 3's basket: 22747

**Dataset Story:**The 'Online Retail II' dataset represents the online sales transactions of a retail company based in the UK from 01/12/2009 to 09/12/2011.The company’s product catalog includes gift items and most of its customers are known to be wholesalers.\

**Variables:**

**InvoiceNo:** Invoice Number (If this code starts with C, it means that the transaction has been cancelled)\
**StockCode:** Product code (unique for each product)\
**Description:** Product name\
**Quantity:** Number of products (How many of the products on the invoices were sold)\
**InvoiceDate:** Invoice date\
**UnitPrice:** Invoice price ( Sterling )\
**CustomerID:** Unique customer number\
**Country:** Country name



## Task 1: Data Preparation
**Step 1:** Load the 2010-2011 sheet from the Online Retail II dataset.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

df_ = pd.read_excel("/kaggle/input/dataset-for-arbrs/online_retail_II.xlsx")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


**Step 2:** Drop observations where StockCode is 'POST'. ('POST' is a charge added to every invoice and does not represent a product.)


In [2]:
df = df[df['StockCode'] != 'POST']

**Step 3:** Drop observations containing missing values.

In [3]:
df.dropna(inplace=True)

**Step 4:** Remove entries from the dataset where Invoice contains 'C' (indicating cancellation).


In [4]:
df = df[~df['Invoice'].astype(str).str.contains('C')]

**Step 5:** Filter out observations where the Price is less than zero.

In [5]:
df = df[df['Price'] > 0]

**Step 6:** Examine and, if necessary, cap outliers in the Price and Quantity variables.


In [6]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

### Task 2: Generating Association Rules for German Customers

**Step 1:** Define the create_invoice_product_df function to create an invoice-product pivot table as follows:


In [7]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

**Step 2:** Define the create_rules function to generate rules and find rules for German customers.


In [8]:
def create_rules(dataframe, id=True, country="Germany"):
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules

df = df_.copy()
df = retail_data_prep(df)
rules = create_rules(df)

/tmp/ipykernel_18/2034261742.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-213.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
/tmp/ipykernel_18/2313223565.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  applymap(lambda x: 1 if x > 0 else 0)
/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


### Task 3: Making Product Recommendations Based on Product IDs in Users' Baskets

**Step 1:** Use the check_id function to find the names of the specified products.

In [9]:
product_id = 21987
check_id(df, product_id)

sorted_rules = rules.sort_values("lift", ascending=False)

recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

recommendation_list[0:3]

['PACK OF 6 SKULL PAPER CUPS']


[21988, 21988, 21988]

**Step 2:** Use the arl_recommender function to make product recommendations for 3 users.


In [10]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]


arl_recommender(rules, 21987, 1)
arl_recommender(rules, 23235, 2)
arl_recommender(rules, 22747, 3)

[22746, 22745, 22745]

**Step 3:** Check the names of the recommended products.

In [11]:
# ['PACK OF 6 SKULL PAPER CUPS']
# Output[17]: [21989, 21988, 21989]
check_id(df, 21989)
# ['PACK OF 20 SKULL PAPER NAPKINS']
check_id(df, 21988)
# ['PACK OF 6 SKULL PAPER PLATES']
check_id(df, 22746)
# ["POPPY'S PLAYHOUSE LIVINGROOM "]
#check_id(df, 23243)
# ['SET OF TEA COFFEE SUGAR TINS PANTRY']

['PACK OF 20 SKULL PAPER NAPKINS']
['PACK OF 6 SKULL PAPER PLATES']
["POPPY'S PLAYHOUSE LIVINGROOM "]
